In [1]:
import csv
import random
import operator
import math

random.seed(47)

In [2]:
def viewDataset(file):
    with open(file) as csvfile:
        lines = csv.reader(csvfile)
        for row in lines:
            print(', '.join(row))


In [3]:
dataset = r'../datasets/iris.data'
viewDataset(dataset)

5.1, 3.5, 1.4, 0.2, Iris-setosa
4.9, 3.0, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.3, 0.2, Iris-setosa
4.6, 3.1, 1.5, 0.2, Iris-setosa
5.0, 3.6, 1.4, 0.2, Iris-setosa
5.4, 3.9, 1.7, 0.4, Iris-setosa
4.6, 3.4, 1.4, 0.3, Iris-setosa
5.0, 3.4, 1.5, 0.2, Iris-setosa
4.4, 2.9, 1.4, 0.2, Iris-setosa
4.9, 3.1, 1.5, 0.1, Iris-setosa
5.4, 3.7, 1.5, 0.2, Iris-setosa
4.8, 3.4, 1.6, 0.2, Iris-setosa
4.8, 3.0, 1.4, 0.1, Iris-setosa
4.3, 3.0, 1.1, 0.1, Iris-setosa
5.8, 4.0, 1.2, 0.2, Iris-setosa
5.7, 4.4, 1.5, 0.4, Iris-setosa
5.4, 3.9, 1.3, 0.4, Iris-setosa
5.1, 3.5, 1.4, 0.3, Iris-setosa
5.7, 3.8, 1.7, 0.3, Iris-setosa
5.1, 3.8, 1.5, 0.3, Iris-setosa
5.4, 3.4, 1.7, 0.2, Iris-setosa
5.1, 3.7, 1.5, 0.4, Iris-setosa
4.6, 3.6, 1.0, 0.2, Iris-setosa
5.1, 3.3, 1.7, 0.5, Iris-setosa
4.8, 3.4, 1.9, 0.2, Iris-setosa
5.0, 3.0, 1.6, 0.2, Iris-setosa
5.0, 3.4, 1.6, 0.4, Iris-setosa
5.2, 3.5, 1.5, 0.2, Iris-setosa
5.2, 3.4, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.6, 0.2, Iris-setosa
4.8, 3.1, 1.6, 0.2, Iris-setosa
5.4, 3.4

In [4]:
def handleDataset(filename, split):
    trainingSet = []
    testSet = []
    with open(filename, 'r') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        for x in range(len(dataset) - 1):
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
            if random.random() < split:
                trainingSet.append(dataset[x])
            else:
                testSet.append(dataset[x])
    return trainingSet, testSet

In [5]:
# test handleDataset

trainingSet, testSet = handleDataset(dataset, 0.66)
print ('Train: ' + repr(len(trainingSet)))
print ('Test: ' + repr(len(testSet)))

Train: 101
Test: 49


In [6]:
def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return math.sqrt(distance)

In [7]:
# Test Euclidean Distance
data1 = [2, 2, 2, 'a']
data2 = [4, 4, 4, 'b']
distance = euclideanDistance(data1, data2, 3)
print('Distance: ' + repr(distance))

Distance: 3.4641016151377544


In [8]:
def getKNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance) - 1
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [9]:
# test getKNeighbors

trainSet = [[2, 2, 2, 'a'], [4, 4, 4, 'b']]
testInstance = [5, 5, 5]
k = 1
neighbors = getKNeighbors(trainSet, testInstance, 1)
print(neighbors)

[[4, 4, 4, 'b']]


In [10]:
def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
#     print(classVotes)
    sortedVotes = sorted(classVotes.items(),
                         key=operator.itemgetter(1), reverse=True)
#     print(sortedVotes)
    return sortedVotes[0][0]

In [11]:
# test getResponse

neighbors = [[1, 1, 1, 'a'], [2, 2, 2, 'a'], [3, 3, 3, 'b']]
print(getResponse(neighbors))

a


In [12]:
def getAccuracy(testSet, predictions):
    correct = 0
    testSet_length = len(testSet)
    for x in range(testSet_length):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/testSet_length) * 100.0

In [13]:
# test getAccuracy

testSet = [[1, 1, 1, 'a'], [2, 2, 2, 'a'], [3, 3, 3, 'b']]
predictions = ['a', 'a', 'a']
accuracy = getAccuracy(testSet, predictions)
print(accuracy)

66.66666666666666


In [14]:
def main():
    # prepare data
    split = 0.67
    trainingSet, testSet = handleDataset(dataset, 0.66)
    print('Train: ' + repr(len(trainingSet)))
    print('Test: ' + repr(len(testSet)))
    # generate predictions
    predictions = []
    k = 3
    for x in range(len(testSet)):
        neighbors = getKNeighbors(trainingSet, testSet[x], k)
        result = getResponse(neighbors)
        predictions.append(result)
        print(f'> predicted = {result}, actual = {testSet[x][-1]}')
                                            
    accuracy = getAccuracy(testSet, predictions)
    print(f'Accuracy: {round(accuracy,3)}%')

In [15]:
main()

Train: 101
Test: 49
> predicted = Iris-setosa, actual = Iris-setosa
> predicted = Iris-setosa, actual = Iris-setosa
> predicted = Iris-setosa, actual = Iris-setosa
> predicted = Iris-setosa, actual = Iris-setosa
> predicted = Iris-setosa, actual = Iris-setosa
> predicted = Iris-setosa, actual = Iris-setosa
> predicted = Iris-setosa, actual = Iris-setosa
> predicted = Iris-setosa, actual = Iris-setosa
> predicted = Iris-setosa, actual = Iris-setosa
> predicted = Iris-setosa, actual = Iris-setosa
> predicted = Iris-setosa, actual = Iris-setosa
> predicted = Iris-setosa, actual = Iris-setosa
> predicted = Iris-setosa, actual = Iris-setosa
> predicted = Iris-setosa, actual = Iris-setosa
> predicted = Iris-setosa, actual = Iris-setosa
> predicted = Iris-setosa, actual = Iris-setosa
> predicted = Iris-setosa, actual = Iris-setosa
> predicted = Iris-setosa, actual = Iris-setosa
> predicted = Iris-setosa, actual = Iris-setosa
> predicted = Iris-setosa, actual = Iris-setosa
> predicted = Iris-v